In [1]:
import numpy as np
import pandas as pd
import imp
import os
import sys
import tensorflow as tf
import pickle
from numba import njit
from functools import lru_cache
from IPython.display import Audio, display
from matplotlib import pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
import random
import torch

import stops

tf.random.set_seed(1234)

imp.reload(stops)


%load_ext line_profiler

# notes
* we should check if we can move state between different lengths of sequences, if so we can train on many differnent bus lines.

In [2]:
url_exception = "http://www.wav-sounds.com/movie/austinpowers.wav"
url_exception = "https://www.myinstants.com/media/sounds/roblox-death-sound_1.mp3"


def play_sound(self, etype, value, tb, tb_offset=None):
    self.showtraceback((etype, value, tb), tb_offset=tb_offset)
    display(Audio(url=url_exception, autoplay=True))


get_ipython().set_custom_exc((Exception,), play_sound)

url_done = (
    "https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav"
)
url_done = "https://www.myinstants.com/media/sounds/taco-bell-bong-sfx.mp3"
url_done = "https://www.myinstants.com/media/sounds/magic_immune.mp3"


def allDone():
    display(
        Audio(
            url=url_done,
            autoplay=True,
        )
    )

In [3]:
data_dir = os.path.join(os.getcwd(), "..", "data")

In [4]:
direction = 1

In [10]:
def get_data(recompute, direction):
    def df_pop(df, labels):
        popped = df[labels]
        df = df.drop(columns=labels)
        return df, popped

    data_dir = os.path.join(os.getcwd(), "..", "data")

    trips, dates, labels = None, None, None
    lstm_dir = os.path.join(data_dir, "trips_for_lstm")
    files = [
        f + "_direction_" + str(direction) + ".npy"
        for f in ["non_categories", "categories", "labels", "cardinality"]
    ]

    cardinality = []

    if recompute:

        pickle_path = os.path.join(data_dir, "mega_pickle")
        df = pd.read_pickle(pickle_path)
        #     df = df.head(100000)

        df = df.drop(
            columns=["route_number", "time_cat_stop_time", "time_cat_leave_time"]
        )

        df = df.sort_values(["service_date", "train", "trip_number", "stop_time"])
        df = df.reset_index(drop=True)

        df = df[df["direction"] == direction]

        dtypes = df.dtypes
        category_names = dtypes[dtypes == "category"]
        category_names = category_names.index.to_list()
        non_category_names = dtypes[dtypes != "category"]
        non_category_names = non_category_names.index.to_list()
        non_category_names.remove("service_date")
        non_category_names.remove("arrival_deviance")
        label_names = ["arrival_deviance"]

        enc = OrdinalEncoder()
        df[category_names] = enc.fit_transform(df[category_names])
        df[category_names] = df[category_names].astype(np.int32)

        for category in category_names:
            cardinality.append(len(df[category].unique()))

        trips = df.groupby(["service_date", "train", "trip_number"])
        trips = [trip for _, trip in trips]

        random.shuffle(
            trips
        )  # WE CAN DO THIS BECAUSE WE'RE ONLY TRYING TO PREDICT THE NEXT TRIP

        hist = {}
        for trip in trips:
            trip = trip.drop_duplicates("location_id")
            s = tuple(trip["location_id"].to_list())
            if s in hist:
                hist[s] = hist[s] + 1
            else:
                hist[s] = 1

        sorted_stops = sorted(hist.items(), key=lambda x: x[1], reverse=True)
        most_common_stop_sequence = list(sorted_stops[0][0])[
            1:-1
        ]  # LAST STOP WAS REGULARLY ~13 MINUTES EARLY...
        cannonical_length = len(most_common_stop_sequence)

        stop_order_dict = {}
        for i, stop in enumerate(most_common_stop_sequence):
            stop_order_dict[stop] = i

        x1 = []
        x2 = []
        y = []
        for trip in trips:
            trip = trip.drop_duplicates("location_id")

            labels = trip[label_names]
            categories = trip[category_names]
            non_categories = trip[non_category_names]

            cats = np.array(categories)
            non_cats = np.array(non_categories)
            l = np.array(labels)

            c_cats = np.zeros((cannonical_length, cats.shape[1]))
            c_non_cats = np.zeros((cannonical_length, non_cats.shape[1]))
            c_l = np.zeros(cannonical_length)

            loc_id_index = category_names.index("location_id")

            this_trips_stops = cats[:, loc_id_index].tolist()

            for i, loc_id in enumerate(most_common_stop_sequence):
                if loc_id in this_trips_stops:
                    idx = this_trips_stops.index(loc_id)
                    c_cats[i] = cats[idx]
                    c_non_cats[i] = non_cats[idx]
                    c_l[i] = l[idx]

            x1.append(c_non_cats)
            x2.append(c_cats)
            y.append(c_l)

        non_categories = np.stack(x1)
        categories = np.stack(x2)
        labels = np.stack(y)
        cardinality = np.array(cardinality)

        arrays = [non_categories, categories, labels, cardinality]

        for file_name, array in zip(files, arrays):
            file_path = os.path.join(lstm_dir, file_name)
            np.save(file_path, array)

    else:
        arrays = []
        for file_name in files:
            file_path = os.path.join(lstm_dir, file_name)
            arrays.append(np.load(file_path))
        non_categories, categories, labels, cardinality = arrays

    return non_categories, categories, labels, cardinality


# non_categories, categories, labels, cardinality = get_data(direction=1, recompute=False)


# print(
#     non_categories.shape,
#     categories.shape,
#     labels.shape,
# )


# datalen = non_categories.shape[0]
# num_stops = non_categories.shape[1]
# non_category_width = non_categories.shape[2]
# category_width = categories.shape[2]

In [17]:
class bus_dataset(torch.utils.data.Dataset):
    def __init__(self, recompute, direction):
        #         self, annotations_file, img_dir, transform=None, target_transform=None
        #     ):

        self.non_categories, self.categories, self.labels, self.cardinality = get_data(
            recompute, direction
        )

        self.datalen = self.non_categories.shape[0]
        self.num_stops = self.non_categories.shape[1]
        self.non_category_width = self.non_categories.shape[2]
        self.category_width = self.categories.shape[2]
        
        self.non_categories_shape = (self.num_stops, self.non_category_width)
        self.categories_shape = (self.num_stops, self.category_width)
        self.label_shape = (self.num_stops,)


    #         self.img_labels = pd.read_csv(annotations_file)
    #         self.img_dir = img_dir
    #         self.transform = transform
    #         self.target_transform = target_transform

    def __len__(self):
        return self.datalen

    #         return len(self.img_labels)

    def __getitem__(self, idx):

        non_category = self.non_categories[idx].copy()
        category = self.categories[idx].copy()
        label = self.labels[idx].copy()

        r = random.randint(0, num_stops)
        non_category[r:] = 0
        category[r:] = 0

        return (non_category, category, r), label


#         img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
#         image = read_image(img_path)
#         label = self.img_labels.iloc[idx, 1]
#         if self.transform:
#             image = self.transform(image)
#         if self.target_transform:
#             label = self.target_transform(label)
#         return image, label

# get the dataset, pull datalen out of it (or just run len) and then you can use the random split thing...

dataset = bus_dataset(recompute=False, direction=1)


batch_size = 32

train = 0.6
val = 0.3
test = 0.1
assert round(train + val + test) == 1

datalen = len(dataset)

train = int(train * datalen)
val = int(val * datalen)
test = datalen - (train + val)

print(train, val, test)


train_data, test_data = torch.utils.data.random_split(dataset, [train + val, test])
train_data, val_data = torch.utils.data.random_split(train_data, [train, val])


train_data = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_data = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

# train_dataset = dataset.take(train).shuffle(10000).batch(batch_size)
# val_dataset = dataset.skip(train).take(val).batch(batch_size)
# test_dataset = dataset.skip(train + val).batch(batch_size)

27521 13760 4588


In [18]:
# non_categories_shape = (num_stops, non_category_width)
# categories_shape = (num_stops, category_width)
# label_shape = (num_stops,)


# num_samples = 4
# datalen = datalen * num_samples
# known_weight = 1


# def data_generator(s):
#     print(f"working on {s}")
#     files = [
#         f + "_direction_" + str(direction) + ".npy"
#         for f in ["non_categories", "categories", "labels", "cardinality"]
#     ]
#     arrays = []
#     for file_name in files:
#         file_path = os.path.join(lstm_dir, file_name)
#         arrays.append(np.load(file_path))
#     non_categories, categories, labels, cardinality = arrays

#     for non_category, category, label in zip(non_categories, categories, labels):
#         samples = random.sample(range(0, num_stops), num_samples)
#         #         samples = sorted(random_list, reverse=True)
#         for i in samples:
#             nc = non_category.copy()
#             c = category.copy()
#             l = label.copy()
#             #             mask = np.ones_like(l)
#             nc[i:] = 0
#             c[i:] = 0
#             #             mask[:i] *= known_weight
#             l[:i] *= known_weight
#             yield {
#                 "non_categories": nc,
#                 "categories": c,
#                 "num_samples": i,
#                 #                 "mask": mask,
#             }, l


# def get_dataset(non_categories, categories, labels):
#     return tf.data.Dataset.from_generator(
#         data_generator,
#         args=[non_categories, categories, labels],
#         output_signature=(
#             {
#                 "non_categories": tf.TensorSpec(
#                     shape=non_categories_shape, dtype=tf.float32
#                 ),
#                 "categories": tf.TensorSpec(shape=categories_shape, dtype=tf.int32),
#                 "num_samples": tf.TensorSpec(shape=(), dtype=tf.int32),
#                 #                 "mask": tf.TensorSpec(shape=label_shape, dtype=tf.int32),
#             },
#             tf.TensorSpec(shape=label_shape, dtype=tf.float32),
#         ),
#     )


# Dataset = tf.data.Dataset
# dataset = Dataset.from_tensor_slices(["Gen_0", "Gen_1", "Gen_2"])
# dataset = ds.interleave(
#     lambda x: Dataset.from_generator(
#         data_generator,
#         args=(x,),
#         output_signature=(
#             {
#                 "non_categories": tf.TensorSpec(
#                     shape=non_categories_shape, dtype=tf.float32
#                 ),
#                 "categories": tf.TensorSpec(shape=categories_shape, dtype=tf.int32),
#                 "num_samples": tf.TensorSpec(shape=(), dtype=tf.int32),
#                 #                 "mask": tf.TensorSpec(shape=label_shape, dtype=tf.int32),
#             },
#             tf.TensorSpec(shape=label_shape, dtype=tf.float32),
#         ),
#     ),
#     cycle_length=3,
#     block_length=1,
#     num_parallel_calls=3,
# )

# Dataset = tf.data.Dataset
# ds = Dataset.from_tensor_slices(['Gen_0', 'Gen_1', 'Gen_2'])
# ds = ds.interleave(lambda x: Dataset.from_generator(py_gen, output_types=(tf.string), args=(x,)),
#                    cycle_length=3,
#                    block_length=1,
#                    num_parallel_calls=3)
# dataset = ds.make_one_shot_iterator().get_next()

In [ ]:
# batch_size = 32

# train = 0.6
# val = 0.4
# test = 0.0
# assert round(train + val + test) == 1

# train = int(train * datalen)
# val = int(val * datalen)
# test = int(test * datalen)

# print(train, val, test)

# # dataset = get_dataset(non_categories, categories, labels)

# train_dataset = dataset.take(train).shuffle(10000).batch(batch_size)
# val_dataset = dataset.skip(train).take(val).batch(batch_size)
# test_dataset = dataset.skip(train + val).batch(batch_size)

In [ ]:
drop_rate = 0.2
lstm_size = 32

# https://stackoverflow.com/questions/56729139/faster-way-to-do-multiple-embeddings-in-pytorch

non_categories_input = tf.keras.Input(shape=non_categories_shape, name="non_categories")
categories_input = tf.keras.Input(shape=categories_shape, name="categories")
# mask_input = tf.keras.Input(shape=label_shape, name="mask")

non_categories = non_categories_input
categories = categories_input
# mask = mask_input

cat_list = tf.unstack(categories, axis=2)
embed_out = []
for cat, card in zip(cat_list, cardinality):
    embed_out.append(tf.keras.layers.Embedding(card, card // 3 + 1)(cat))

lstm_prep = tf.keras.layers.Concatenate(axis=2)([non_categories, *embed_out])

concat_width = lstm_prep.shape[2]

lstm_prep = tf.keras.layers.Dense((lstm_size + concat_width) // 2 + 1)(lstm_prep)
lstm_prep = tf.keras.layers.Dense(lstm_size)(lstm_prep)
block_out = tf.keras.layers.Reshape((num_stops, lstm_size))(lstm_prep)


lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

lstm_layers = tf.keras.layers.BatchNormalization()(lstm_layers)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

block_out = tf.keras.layers.add([block_out, lstm_layers])

lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

lstm_layers = tf.keras.layers.BatchNormalization()(lstm_layers)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

block_out = tf.keras.layers.add([block_out, lstm_layers])

lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

lstm_layers = tf.keras.layers.BatchNormalization()(lstm_layers)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

block_out = tf.keras.layers.add([block_out, lstm_layers])


lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

lstm_layers = tf.keras.layers.BatchNormalization()(lstm_layers)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

block_out = tf.keras.layers.add([block_out, lstm_layers])


lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

lstm_layers = tf.keras.layers.BatchNormalization()(lstm_layers)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

block_out = tf.keras.layers.add([block_out, lstm_layers])


lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)

out_funnel = tf.keras.layers.Dense(1)

pred = tf.keras.layers.TimeDistributed(out_funnel)(lstm_layers)

pred = tf.keras.layers.Reshape(label_shape)(pred)
# pred = pred * mask

model = tf.keras.Model(
    inputs=[non_categories_input, categories],
    outputs=[pred],
)

In [ ]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"),
    metrics=["MAE"],
)
print("total parameters: {:,}".format(model.count_params()))
tf.keras.utils.plot_model(model, "model.png", show_shapes=True, dpi=227)

In [11]:
monitor_param = "val_loss"

nan_terminate = tf.keras.callbacks.TerminateOnNaN()

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor=monitor_param, patience=10  # , restore_best_weights=True
)

model_dir = os.path.join(data_dir, "model")
checkpoint_filepath = os.path.join(model_dir, "checkpoint")
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor=monitor_param,
    mode="min",
    save_best_only=True,
)

In [12]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

model.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset,
    callbacks=[early_stopping, checkpoint, nan_terminate],
)

Num GPUs Available:  1
Epoch 1/100


C:\Users\graham\AppData\Local\pypoetry\Cache\virtualenvs\bus-prediction-iVe0Nv0y-py3.8\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['num_samples'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


3441/3441 [==============================] - 356s 99ms/step - loss: 26170.1562 - MAE: 85.5732 - val_loss: 26518.2090 - val_MAE: 86.4914
Epoch 2/100
3441/3441 [==============================] - ETA: 0s - loss: 25890.3281 - MAE: 85.9101

KeyboardInterrupt: 

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
# model.evaluate(test_dataset)

In [ ]:
preds = model.predict(test_dataset)

In [ ]:
print(preds.shape)

In [ ]:
for i, (pred, y) in enumerate(zip(preds, test_dataset.unbatch().as_numpy_iterator())):
    if i < 100:
        x, y = y
        pred_len = pred.shape[0]

        num_samples = x["num_samples"]
        samp = [0] * num_samples

        x = list(range(pred_len))
        #         print(pred)

        plt.rcParams["figure.figsize"] = [20, 4]
        plt.scatter(x, pred, label="pred")
        plt.scatter(x, y, label="y")
        plt.scatter(list(range(num_samples)), samp, label="samp")
        plt.legend()
        plt.show()

allDone()